In [1]:
import findspark
findspark.init() 

In [2]:
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
# May take awhile locally
spark = SparkSession.builder.appName("PySpark").getOrCreate()
spark

#### Create a Spark dataframe
In PySpark you need to create a Spark session first before you do anything. Then the createdataframe is inherent in session.

In [3]:
data = [['tom', 10], ['nick', 15], ['juli', 14]]
df = spark.createDataFrame(data,['Name','Age'])

In [4]:
df.show()

+----+---+
|Name|Age|
+----+---+
| tom| 10|
|nick| 15|
|juli| 14|
+----+---+



In [5]:
df.toPandas()

,Name,Age
0,tom,10
1,nick,15
2,juli,14


In [6]:
df.columns

['Name', 'Age']

In [8]:
df.count()

3

In [10]:
path = 'students.csv'
df = spark.read.csv(path,header = True)
df.show()

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|female|       group B|          bachelor's degree|    standard|                   none|        72|           72|           74|
|female|       group C|               some college|    standard|              completed|        69|           90|           88|
|female|       group B|            master's degree|    standard|                   none|        90|           95|           93|
|  male|       group A|         associate's degree|free/reduced|                   none|        47|           57|           44|
|  male|       group C|               some college|    standard|                   none|        76|     

In [11]:
df.toPandas()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75
...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,88,99,95
996,male,group C,high school,free/reduced,none,62,55,55
997,female,group C,high school,free/reduced,completed,59,71,65
998,female,group D,some college,standard,completed,68,78,77


In [12]:
df.groupBy('gender').agg({'math score':'mean'}).show()

+------+------------------+
|gender|   avg(math score)|
+------+------------------+
|female|63.633204633204635|
|  male| 68.72821576763485|
+------+------------------+



In [13]:
from pyspark.sql import functions as F
df.groupBy("gender").agg(F.min("math score"), F.max("math score"), F.avg("math score")).show()

+------+---------------+---------------+------------------+
|gender|min(math score)|max(math score)|   avg(math score)|
+------+---------------+---------------+------------------+
|female|              0|             99|63.633204633204635|
|  male|            100|             99| 68.72821576763485|
+------+---------------+---------------+------------------+



### Sparks Immutability

- Spark DataFrames are built on top of RDDs which are immutable in nature, hence Data frames are immutable in nature as well
- So if you make a change to a dataframe like adding a column or changing any of the values in the dataframe using the same naming convention, it will generate a new dataframe (with a new unique ID) instead of updating the existing data frame

In [14]:
# Let's fetch the id of our dataframe we created above
df.rdd.id()

59

In [15]:
# Even if we duplicate the dataframe, the ID remains the same
df2 = df
df2.rdd.id()

59

In [16]:
df = df.withColumn('new_col', df['math score'] * 2)
df.rdd.id()

65

In [18]:
df.toPandas()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score,new_col
0,female,group B,bachelor's degree,standard,none,72,72,74,144.0
1,female,group C,some college,standard,completed,69,90,88,138.0
2,female,group B,master's degree,standard,none,90,95,93,180.0
3,male,group A,associate's degree,free/reduced,none,47,57,44,94.0
4,male,group C,some college,standard,none,76,78,75,152.0
...,...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,88,99,95,176.0
996,male,group C,high school,free/reduced,none,62,55,55,124.0
997,female,group C,high school,free/reduced,completed,59,71,65,118.0
998,female,group D,some college,standard,completed,68,78,77,136.0


- Putting your data in spark dataframe allows spark to distribute your data as you work with it
- PySpark also offers a built in library to read and write partitioned datasets which is preferred storage method that divides that dataset into more than one part
- When it comes to bigdata management,the prefereed method is to partition files and store them as parquet files

### Reading Writing and Validating Data in PySpark

In [19]:
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
# May take awhile locally
spark = SparkSession.builder.appName("ReadWriteVal").getOrCreate()

cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")
spark

You are working with 1 core(s)


In [20]:
path = 'students.csv'
students = spark.read.csv(path,header = True,inferSchema = True)
students.show(5)

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|female|       group B|          bachelor's degree|    standard|                   none|        72|           72|           74|
|female|       group C|               some college|    standard|              completed|        69|           90|           88|
|female|       group B|            master's degree|    standard|                   none|        90|           95|           93|
|  male|       group A|         associate's degree|free/reduced|                   none|        47|           57|           44|
|  male|       group C|               some college|    standard|                   none|        76|     

In [21]:
students.limit(4).toPandas()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44


In [24]:
path = "Read+Write+Validate+Datasets/"
parquet = spark.read.parquet(path+'users1.parquet')
parquet.show(5)

+-------------------+---+----------+---------+--------------------+------+--------------+----------------+------------+---------+---------+--------------------+--------+
|  registration_dttm| id|first_name|last_name|               email|gender|    ip_address|              cc|     country|birthdate|   salary|               title|comments|
+-------------------+---+----------+---------+--------------------+------+--------------+----------------+------------+---------+---------+--------------------+--------+
|2016-02-03 13:25:29|  1|    Amanda|   Jordan|    ajordan0@com.com|Female|   1.197.201.2|6759521864920116|   Indonesia| 3/8/1971| 49756.53|    Internal Auditor|   1E+02|
|2016-02-03 22:34:03|  2|    Albert|  Freeman|     afreeman1@is.gd|  Male|218.111.175.34|                |      Canada|1/16/1968|150280.17|       Accountant IV|        |
|2016-02-03 06:39:31|  3|    Evelyn|   Morgan|emorgan2@altervis...|Female|  7.161.136.94|6767119071901597|      Russia| 2/1/1960|144972.51| Structural

In [25]:
parquet.limit(5).toPandas()

,registration_dttm,id,first_name,last_name,email,gender,ip_address,cc,country,birthdate,salary,title,comments
0,2016-02-03 13:25:29,1,Amanda,Jordan,ajordan0@com.com,Female,1.197.201.2,6759521864920116,Indonesia,3/8/1971,49756.53,Internal Auditor,1E+02
1,2016-02-03 22:34:03,2,Albert,Freeman,afreeman1@is.gd,Male,218.111.175.34,,Canada,1/16/1968,150280.17,Accountant IV,
2,2016-02-03 06:39:31,3,Evelyn,Morgan,emorgan2@altervista.org,Female,7.161.136.94,6767119071901597,Russia,2/1/1960,144972.51,Structural Engineer,
3,2016-02-03 06:06:21,4,Denise,Riley,driley3@gmpg.org,Female,140.35.109.83,3576031598965625,China,4/8/1997,90263.05,Senior Cost Accountant,
4,2016-02-03 10:35:31,5,Carlos,Burns,cburns4@miitbeian.gov.cn,,169.113.235.40,5602256255204850,South Africa,,NaN,,


In [26]:
partitioned = spark.read.parquet(path+'users*')
partitioned.show(2)

+-------------------+---+----------+---------+----------------+------+--------------+----------------+---------+---------+---------+----------------+--------+
|  registration_dttm| id|first_name|last_name|           email|gender|    ip_address|              cc|  country|birthdate|   salary|           title|comments|
+-------------------+---+----------+---------+----------------+------+--------------+----------------+---------+---------+---------+----------------+--------+
|2016-02-03 13:25:29|  1|    Amanda|   Jordan|ajordan0@com.com|Female|   1.197.201.2|6759521864920116|Indonesia| 3/8/1971| 49756.53|Internal Auditor|   1E+02|
|2016-02-03 22:34:03|  2|    Albert|  Freeman| afreeman1@is.gd|  Male|218.111.175.34|                |   Canada|1/16/1968|150280.17|   Accountant IV|        |
+-------------------+---+----------+---------+----------------+------+--------------+----------------+---------+---------+---------+----------------+--------+
only showing top 2 rows



In [28]:
# Note that the .option("basePath", path) option is used to override the automatic function
# that will exclude the partitioned variable in resulting dataframe. 
# I prefer to have the partitioning info in my new dataframe personally. 
users1_2 = spark.read.option("basePath", path).parquet(path+'users1.parquet', path+'users2.parquet')
users1_2.show(4)

+-------------------+---+----------+---------+--------------------+------+--------------+----------------+---------+---------+---------+--------------------+--------+
|  registration_dttm| id|first_name|last_name|               email|gender|    ip_address|              cc|  country|birthdate|   salary|               title|comments|
+-------------------+---+----------+---------+--------------------+------+--------------+----------------+---------+---------+---------+--------------------+--------+
|2016-02-03 13:25:29|  1|    Amanda|   Jordan|    ajordan0@com.com|Female|   1.197.201.2|6759521864920116|Indonesia| 3/8/1971| 49756.53|    Internal Auditor|   1E+02|
|2016-02-03 22:34:03|  2|    Albert|  Freeman|     afreeman1@is.gd|  Male|218.111.175.34|                |   Canada|1/16/1968|150280.17|       Accountant IV|        |
|2016-02-03 06:39:31|  3|    Evelyn|   Morgan|emorgan2@altervis...|Female|  7.161.136.94|6767119071901597|   Russia| 2/1/1960|144972.51| Structural Engineer|        

### Validating Data

In [29]:
# Get an inital view of your dataframe
students.show(3)

+------+--------------+---------------------------+--------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental level of education|   lunch|test preparation course|math score|reading score|writing score|
+------+--------------+---------------------------+--------+-----------------------+----------+-------------+-------------+
|female|       group B|          bachelor's degree|standard|                   none|        72|           72|           74|
|female|       group C|               some college|standard|              completed|        69|           90|           88|
|female|       group B|            master's degree|standard|                   none|        90|           95|           93|
+------+--------------+---------------------------+--------+-----------------------+----------+-------------+-------------+
only showing top 3 rows



In [30]:
# If your dataframe is more than just a few variables, this method is way better
students.limit(5).toPandas()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [31]:
print(students.printSchema())
print("")
print(students.columns)
print("")
print(students.describe())

root
 |-- gender: string (nullable = true)
 |-- race/ethnicity: string (nullable = true)
 |-- parental level of education: string (nullable = true)
 |-- lunch: string (nullable = true)
 |-- test preparation course: string (nullable = true)
 |-- math score: integer (nullable = true)
 |-- reading score: integer (nullable = true)
 |-- writing score: integer (nullable = true)

None

['gender', 'race/ethnicity', 'parental level of education', 'lunch', 'test preparation course', 'math score', 'reading score', 'writing score']

DataFrame[summary: string, gender: string, race/ethnicity: string, parental level of education: string, lunch: string, test preparation course: string, math score: string, reading score: string, writing score: string]


In [32]:
# If you need to get the type of just ONE column by name you can use this function:
students.schema['math score'].dataType

IntegerType

In [33]:
students.describe(['math score']).show()

+-------+------------------+
|summary|        math score|
+-------+------------------+
|  count|              1000|
|   mean|            66.089|
| stddev|15.163080096009454|
|    min|                 0|
|    max|               100|
+-------+------------------+



In [34]:
students.select("math score", "reading score","writing score").summary("count", "min", "25%", "75%", "max").show()

+-------+----------+-------------+-------------+
|summary|math score|reading score|writing score|
+-------+----------+-------------+-------------+
|  count|      1000|         1000|         1000|
|    min|         0|           17|           10|
|    25%|        57|           59|           57|
|    75%|        77|           79|           79|
|    max|       100|          100|          100|
+-------+----------+-------------+-------------+



### How to specify data types as you read in datasets.

In [35]:
from pyspark.sql.types import *

In [36]:
data_schema = [StructField("name", StringType(), True),
               StructField("email", StringType(), True),
               StructField("city", StringType(), True),
               StructField("mac", StringType(), True),
               StructField("timestamp", DateType(), True),
               StructField("creditcard", StringType(), True)]

In [37]:
final_struc = StructType(fields=data_schema)

In [38]:
path = "Read+Write+Validate+Datasets/"
people = spark.read.json(path+'people.json', schema=final_struc)

In [39]:
people.printSchema()

root
 |-- name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- city: string (nullable = true)
 |-- mac: string (nullable = true)
 |-- timestamp: date (nullable = true)
 |-- creditcard: string (nullable = true)



### Writing Data

In [43]:
students.write.mode("overwrite").csv('write_test.csv')

- Spark uses Hadoop File Format, which requires data to be partitioned - that's why you have part- files.

In [41]:
from py4j.java_gateway import java_import
java_import(spark._jvm, 'org.apache.hadoop.fs.Path')

fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())
file = fs.globStatus(spark._jvm.Path('write_test.csv/part*'))[0].getPath().getName()
fs.rename(spark._jvm.Path('write_test.csv/' + file), spark._jvm.Path('write_test2.csv')) #these two need to be different
fs.delete(spark._jvm.Path('write_test.csv'), True)

True

#### Writting Parquet files

In [44]:
users1_2.write.mode("overwrite").parquet('parquet1/')

#### Writting Partitioned Parquet Files

In [45]:
users1_2.write.mode("overwrite").partitionBy("gender").parquet('part_parquet/')

#### Writting your dataframes

In [46]:
values = [('Pear',10),('Orange',36),('Banana',123),('Kiwi',48),('Peach',16),('Strawberry',1)]
df = spark.createDataFrame(values,['fruit','quantity'])
df.show()

+----------+--------+
|     fruit|quantity|
+----------+--------+
|      Pear|      10|
|    Orange|      36|
|    Banana|     123|
|      Kiwi|      48|
|     Peach|      16|
|Strawberry|       1|
+----------+--------+



### Search and Filter DataFrames in PySpark

In [2]:
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
# May take awhile locally
spark = SparkSession.builder.appName("Select").getOrCreate()

cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")
spark

You are working with 1 core(s)


In [3]:
fifa = spark.read.csv('fifaa19.csv',inferSchema=True,header=True)

In [4]:
fifa.limit(4).toPandas()

,_c0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,...,96,33,28,26,6,11,15,14,8,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,...,95,28,31,23,7,11,15,14,11,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,...,94,27,24,33,9,9,15,15,11,€228.1M
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,...,68,15,21,13,90,85,87,88,94,€138.6M


In [5]:
print(fifa.printSchema())

root
 |-- _c0: integer (nullable = true)
 |-- ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Photo: string (nullable = true)
 |-- Nationality: string (nullable = true)
 |-- Flag: string (nullable = true)
 |-- Overall: integer (nullable = true)
 |-- Potential: integer (nullable = true)
 |-- Club: string (nullable = true)
 |-- Club Logo: string (nullable = true)
 |-- Value: string (nullable = true)
 |-- Wage: string (nullable = true)
 |-- Special: integer (nullable = true)
 |-- Preferred Foot: string (nullable = true)
 |-- International Reputation: integer (nullable = true)
 |-- Weak Foot: integer (nullable = true)
 |-- Skill Moves: integer (nullable = true)
 |-- Work Rate: string (nullable = true)
 |-- Body Type: string (nullable = true)
 |-- Real Face: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Jersey Number: integer (nullable = true)
 |-- Joined: string (nullable = true)
 |-- Loaned From: string (nu

In [6]:
fifa.select(['Nationality','Name','Age']).show(5)

+-----------+-----------------+---+
|Nationality|             Name|Age|
+-----------+-----------------+---+
|  Argentina|         L. Messi| 31|
|   Portugal|Cristiano Ronaldo| 33|
|     Brazil|        Neymar Jr| 26|
|      Spain|           De Gea| 27|
|    Belgium|     K. De Bruyne| 27|
+-----------+-----------------+---+
only showing top 5 rows



In [7]:
fifa.select(['Nationality','Name','Age','Photo']).show(5,False)

+-----------+-----------------+---+----------------------------------------------+
|Nationality|Name             |Age|Photo                                         |
+-----------+-----------------+---+----------------------------------------------+
|Argentina  |L. Messi         |31 |https://cdn.sofifa.org/players/4/19/158023.png|
|Portugal   |Cristiano Ronaldo|33 |https://cdn.sofifa.org/players/4/19/20801.png |
|Brazil     |Neymar Jr        |26 |https://cdn.sofifa.org/players/4/19/190871.png|
|Spain      |De Gea           |27 |https://cdn.sofifa.org/players/4/19/193080.png|
|Belgium    |K. De Bruyne     |27 |https://cdn.sofifa.org/players/4/19/192985.png|
+-----------+-----------------+---+----------------------------------------------+
only showing top 5 rows



In [8]:
# who is the youngest player in the dataset?
fifa.select(['Nationality','Name','Age']).orderBy("Age").show(5)

+-------------+------------+---+
|  Nationality|        Name|Age|
+-------------+------------+---+
|       Sweden|   B. Nygren| 16|
|       Sweden|H. Andersson| 16|
|       Turkey|    A. Doğan| 16|
|United States|  C. Bassett| 16|
|      England|    B. Mumba| 16|
+-------------+------------+---+
only showing top 5 rows



In [9]:
fifa.select(['Nationality','Name','Age']).orderBy(fifa["Age"].desc()).show(5)

+-----------------+-------------+---+
|      Nationality|         Name|Age|
+-----------------+-------------+---+
|           Mexico|     O. Pérez| 45|
|          England|K. Pilkington| 44|
|Trinidad & Tobago|    T. Warner| 44|
|            Japan|  S. Narazaki| 42|
|         Paraguay|    J. Villar| 41|
+-----------------+-------------+---+
only showing top 5 rows



In [10]:
fifa.select("Name","Club").where(fifa.Club.like("%Barcelona%")).show(5, False)

+---------------+------------+
|Name           |Club        |
+---------------+------------+
|L. Messi       |FC Barcelona|
|L. Suárez      |FC Barcelona|
|M. ter Stegen  |FC Barcelona|
|Sergio Busquets|FC Barcelona|
|Coutinho       |FC Barcelona|
+---------------+------------+
only showing top 5 rows



In [11]:
# Select last 4 characters of the photo column to understand all file types used
# This says return 
fifa.select("Photo",fifa.Photo.substr(-4,4)).show(5,False)

+----------------------------------------------+-----------------------+
|Photo                                         |substring(Photo, -4, 4)|
+----------------------------------------------+-----------------------+
|https://cdn.sofifa.org/players/4/19/158023.png|.png                   |
|https://cdn.sofifa.org/players/4/19/20801.png |.png                   |
|https://cdn.sofifa.org/players/4/19/190871.png|.png                   |
|https://cdn.sofifa.org/players/4/19/193080.png|.png                   |
|https://cdn.sofifa.org/players/4/19/192985.png|.png                   |
+----------------------------------------------+-----------------------+
only showing top 5 rows



In [12]:
fifa.select("Photo",fifa.Photo.substr(32, 11)).show(5,False)

+----------------------------------------------+------------------------+
|Photo                                         |substring(Photo, 32, 11)|
+----------------------------------------------+------------------------+
|https://cdn.sofifa.org/players/4/19/158023.png|4/19/158023             |
|https://cdn.sofifa.org/players/4/19/20801.png |4/19/20801.             |
|https://cdn.sofifa.org/players/4/19/190871.png|4/19/190871             |
|https://cdn.sofifa.org/players/4/19/193080.png|4/19/193080             |
|https://cdn.sofifa.org/players/4/19/192985.png|4/19/192985             |
+----------------------------------------------+------------------------+
only showing top 5 rows



In [13]:
fifa[fifa.Club.isin("FC Barcelona","Juventus")].limit(4).toPandas()

,_c0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,...,96,33,28,26,6,11,15,14,8,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,...,95,28,31,23,7,11,15,14,11,€127.1M
2,7,176580,L. Suárez,31,https://cdn.sofifa.org/players/4/19/176580.png,Uruguay,https://cdn.sofifa.org/flags/60.png,91,91,FC Barcelona,...,85,62,45,38,27,25,31,33,37,€164M
3,15,211110,P. Dybala,24,https://cdn.sofifa.org/players/4/19/211110.png,Argentina,https://cdn.sofifa.org/flags/52.png,89,94,Juventus,...,84,23,20,20,5,4,4,5,8,€153.5M


In [14]:
fifa.select("Name","Club").where(fifa.Name.startswith("L")) \
                                  .where(fifa.Name.endswith("i")).limit(4).toPandas()

,Name,Club
0,L. Messi,FC Barcelona
1,L. Bonucci,Juventus
2,L. Fabiański,West Ham United
3,L. Pellegrini,Roma


In [15]:
fifa.count()

18207

In [16]:
df = fifa.limit(100)
df.count()

100

In [17]:
col_list = fifa.columns[0:5]
df3 = fifa.select(col_list)

In [19]:
df3.show(5)

+---+------+-----------------+---+--------------------+
|_c0|    ID|             Name|Age|               Photo|
+---+------+-----------------+---+--------------------+
|  0|158023|         L. Messi| 31|https://cdn.sofif...|
|  1| 20801|Cristiano Ronaldo| 33|https://cdn.sofif...|
|  2|190871|        Neymar Jr| 26|https://cdn.sofif...|
|  3|193080|           De Gea| 27|https://cdn.sofif...|
|  4|192985|     K. De Bruyne| 27|https://cdn.sofif...|
+---+------+-----------------+---+--------------------+
only showing top 5 rows



#### PySpark slicing and filtering

- In PySpark index starts with 1

In [20]:
from pyspark.sql.functions import slice

df = spark.createDataFrame([([1, 2, 3],), ([4, 5],)], ['x']) 
df.show()
df.select(slice(df.x, 2, 2).alias("sliced")).show()

+---------+
|        x|
+---------+
|[1, 2, 3]|
|   [4, 5]|
+---------+

+------+
|sliced|
+------+
|[2, 3]|
|   [5]|
+------+



In [21]:
fifa.filter("Overall > 50").limit(5).toPandas()

,_c0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,...,96,33,28,26,6,11,15,14,8,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,...,95,28,31,23,7,11,15,14,11,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,...,94,27,24,33,9,9,15,15,11,€228.1M
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,...,68,15,21,13,90,85,87,88,94,€138.6M
4,4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,...,88,68,58,51,15,13,5,10,13,€196.4M


In [22]:
fifa.filter("Overall>50").select(['Name','Age']).limit(5).toPandas()

,Name,Age
0,L. Messi,31
1,Cristiano Ronaldo,33
2,Neymar Jr,26
3,De Gea,27
4,K. De Bruyne,27


In [24]:
fifa.select(['Nationality','Name','Age','Overall']).filter("Overall>70").orderBy(fifa["Overall"].desc()).show(5)

+-----------+-----------------+---+-------+
|Nationality|             Name|Age|Overall|
+-----------+-----------------+---+-------+
|   Portugal|Cristiano Ronaldo| 33|     94|
|  Argentina|         L. Messi| 31|     94|
|     Brazil|        Neymar Jr| 26|     92|
|      Spain|           De Gea| 27|     91|
|    Belgium|     K. De Bruyne| 27|     91|
+-----------+-----------------+---+-------+
only showing top 5 rows



#### Collecting Results as Objects

In [25]:
result = fifa.select(['Nationality','Name','Age','Overall']).filter("Overall>70").orderBy(fifa["Overall"].desc()).collect()

In [26]:
result

[Row(Nationality='Argentina', Name='L. Messi', Age=31, Overall=94),
 Row(Nationality='Portugal', Name='Cristiano Ronaldo', Age=33, Overall=94),
 Row(Nationality='Brazil', Name='Neymar Jr', Age=26, Overall=92),
 Row(Nationality='Spain', Name='De Gea', Age=27, Overall=91),
 Row(Nationality='Belgium', Name='K. De Bruyne', Age=27, Overall=91),
 Row(Nationality='Belgium', Name='E. Hazard', Age=27, Overall=91),
 Row(Nationality='Croatia', Name='L. Modrić', Age=32, Overall=91),
 Row(Nationality='Uruguay', Name='L. Suárez', Age=31, Overall=91),
 Row(Nationality='Spain', Name='Sergio Ramos', Age=32, Overall=91),
 Row(Nationality='Slovenia', Name='J. Oblak', Age=25, Overall=90),
 Row(Nationality='Poland', Name='R. Lewandowski', Age=29, Overall=90),
 Row(Nationality='Germany', Name='T. Kroos', Age=28, Overall=90),
 Row(Nationality='Uruguay', Name='D. Godín', Age=32, Overall=90),
 Row(Nationality='Spain', Name='David Silva', Age=32, Overall=90),
 Row(Nationality='France', Name='N. Kanté', Age=27, 

In [27]:
print("Best Player Over 70: ",result[0][1])
print("Nationality of Best Player Over 70: ",result[0][0])
print("")
print("Worst Player Over 70: ",result[-1][1])
print("Nationality of Worst Player Over 70: ",result[-1][0])

Best Player Over 70:  L. Messi
Nationality of Best Player Over 70:  Argentina

Worst Player Over 70:  Zapater
Nationality of Worst Player Over 70:  Spain


In [28]:
for item in result[0]:
    print(item)

Argentina
L. Messi
31
94
